### Supervisor architecture applied to Genie and RAG
 - Genie within tool-calling agent
 - Genie as standalone agent


 

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from chatagent import ChatModel

In [0]:
from agents.supervisor.graph import app as supervisor

In [0]:
import mlflow
from typing import Annotated, TypedDict
from typing import Literal, Any
from langgraph.types import Command
from mlflow.langchain.chat_agent_langgraph import ChatAgentState
from langgraph.graph import START, END, StateGraph
from agents.genie.graph import app as genie_agent
from agents.rag.graph import app as rag_agent
from agents.supervisor.resources.model import model

config = mlflow.models.ModelConfig(development_config="config.yaml")
genie_config = config.get("agents").get("genie")[0]

member_agents = ["rag", "genie"]
options = member_agents + ["FINISH"]

class Router(TypedDict):
    next: Literal[*options]


system_prompt = f"""You are a supervisor tasked with managing a conversation between the following workers: {member_agents}. The worker's capabilities are as follows.
                 
    - genie: Performs text to SQL against supply chain data tables. 
    - rag: Retrieves documentation related to Databricks and Apache Spark.

Given the following user request respond with the worker to act next. Each worker will perform a task and respond with their results and status. Call only the worker(s) that are important to the user's qeustion. Do not provide insights or summaries of the data returned by the workes. Your only job is to return the work names only. When finished respond with FINISH."""


def supervisor_node(state: ChatAgentState) -> Command[Literal[*member_agents, "__end__"]]:
    messages = [{"role": "system", "content": system_prompt}] + state['messages']
    supervisor_model = model.with_structured_output(Router)
    response = supervisor_model.invoke(messages)
    goto = response["next"]
    #goto = response.content
    if goto == "FINISH":
        goto = END
    return Command(goto=goto, update={"next": goto})
  

def rag_node(state: ChatAgentState):
  response = rag_agent.invoke(state)
  return Command(
        update={"messages": response['messages']},
        goto="supervisor",
    )


def genie_node(state: ChatAgentState):
  response = genie_agent.invoke(state)
  return Command(
        update={"messages": response['messages']},
        goto="supervisor",
    )
  

workflow = StateGraph(ChatAgentState)
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("genie", genie_node)
workflow.add_node("rag", rag_node)
workflow.add_edge(START, "supervisor")
supervisor = workflow.compile()

In [0]:
message = {"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages?"}]}
query_results = supervisor.invoke(message)

In [0]:
message = {"messages": [{"role": "user", "content": "What is Apache Spark?"}]}
query_results = supervisor.invoke(message)

In [0]:
message = {"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages? Can you also tell me what Apache Spark is?"}]}
query_results = supervisor.invoke(message)

In [0]:
def call_genie(state: ChatAgentState):
  result = genie_agent.invoke(state)
  return {
        "messages": [
            {
                "role": "assistant",
                "content": result["messages"][-1].content,
                "name": genie_config['name'],
            }
        ]
    }
  
graph_builder = StateGraph(ChatAgentState)
graph_builder.add_node("Genie", call_genie)
graph_builder.add_edge(START, "Genie")
graph_builder.add_edge("Genie", END)
app = graph_builder.compile()

In [0]:
follow_up = {"messages": query_results['messages'] + [{"role": "user", "content": "What about the top 10?"}]}
follow_up_results = app.invoke(follow_up)
             

In [0]:
query_results['messages'][0].content

In [0]:
from agents.genie.resources.genie_room import genie_agent

message = {"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages?"}]}
query_results = genie_agent.invoke(message)

In [0]:
class Router(TypedDict):
    next: Literal[*options]

question = [{"role": "user", "content": "What are our top 3 forecasted raw material shortages?"}]
messages = [{"role": "system", "content": system_prompt}] + question

supervisor_model = model.with_structured_output(Router)
next_response = model.invoke(messages)

In [0]:
next_response.content

In [0]:
member_agents = ["rag", "genie"]
options = member_agents + ["FINISH"]

system_prompt = f"""You are a supervisor tasked with managing a conversation between the following workers: {member_agents}. The worker's capabilities are as follows.
                 
    - genie: Performs text to SQL against supply chain data tables. 
    - rag: Retrieves documentation related to Databricks and Apache Spark.

Given the following user request respond with the worker to act next. Each worker will perform a task and respond with their results and status. Call only the worker(s) that are important to the user's qeustion. Do not provide insights or summaries of the data returned by the workes. Your only job is to return the work names only. When finished respond with FINISH."""





def supervisor_node(state: ChatAgentState) -> Command[Literal[*member_agents, "__end__"]]:
    messages = [{"role": "system", "content": system_prompt}] + state['messages']
    supervisor_model = model.with_structured_output(Router)
    response = supervisor_model.invoke(messages)
    goto = response["next"]
    #goto = response.content
    if goto == "FINISH":
        goto = END
    return Command(goto=goto, update={"next": goto})
  

def rag_node(state: ChatAgentState):
  response = rag_agent.invoke(state)
  return Command(
        update={"messages": response['messages']},
        goto="supervisor",
    )


def genie_node(state: ChatAgentState):
  response = genie_agent.invoke(state)
  return Command(
        update={"messages": response['messages']},
        goto="supervisor",
    )
  
#workflow.add_node("final_answer", final_answer)

workflow = StateGraph(ChatAgentState)
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("genie", genie_node)
workflow.add_node("rag", rag_node)
workflow.add_edge(START, "supervisor")
supervisor = workflow.compile()

In [0]:
supervisor_results = supervisor.invoke({"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages?. Can. you also tell me if Apache Spark supports streaming?"}]})

In [0]:
from agents.genie.resources.genie_room import genie_agent

query_results = genie_agent.invoke({"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages?"}]})

In [0]:
genie_result = genie_agent.invoke({"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages"}]})

In [0]:
genie_result['messages']

In [0]:
genie_messages = {"messages": genie_result['messages'] + 
                         [{"role": "user", "content": "What about the top 5?"}]}

genie_follow_up = genie_agent.invoke(genie_messages)

In [0]:
rag_result = rag_agent.invoke({"messages": [{"role": "user", 
                                             "content": "Does apache spark support streaming?"}]})

In [0]:
messages = {"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages"}, 
                         {"role": "assistant", "content": result['messages'][0].content},
                         {"role": "user", "content": "What about the top 5?"}]}

follow_up = genie_agent.invoke(messages)

In [0]:
from agents.genie.graph import app as genie_agent

In [0]:
genie_agent.invoke({"messages": [{"role": "user", "content": "What are our top 3 forecasted raw material shortages"}]})